<a href="https://colab.research.google.com/github/Natural-State/agol-data-workflows/blob/master/code/Colab%20notebooks/02_MOD13Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extract NDVI & EVI for an AOI and export to Google Drive

## Import gee and authenticate

In [ ]:
import ee

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

## Input arguments for data extraction

In [ ]:
# Area of interest
aoi = ee.FeatureCollection("projects/ns-agol-rs-data/assets/LLBN")
aoi_name = "LLBN"

# GEE layer ID (options: NDVI, EVI)
# band_layer = "NDVI"
# layer_name = "RS_002"

band_layer = "EVI"
layer_name = "RS_003"

# Image reducer (options: mean, median, min, max, stdDev, sum, product)
img_col_reducer = "mean"

# Date parameters
start_year = 2012
end_year = 2022

# Range doesn't include the stop value
year_list = ee.List(list(range(start_year, end_year+1)))

# Season parameters (months)
rain_start = 3
rain_end = 5
dry_start = 7
dry_end = 10

## Import MOD13Q1.061 Terra Vegetation Indices image collection



In [ ]:
modis = ee.ImageCollection("MODIS/061/MOD13Q1")

## NDVI/EVI processing

In [ ]:
reducer_list = ee.Reducer.mean() \
.combine(reducer2 = ee.Reducer.median(), sharedInputs=True) \
.combine(reducer2 = ee.Reducer.min(), sharedInputs=True) \
.combine(reducer2 = ee.Reducer.max(), sharedInputs=True) \
.combine(reducer2 = ee.Reducer.stdDev(), sharedInputs=True) \
.combine(reducer2 = ee.Reducer.sum(), sharedInputs=True) \
.combine(reducer2 = ee.Reducer.product(), sharedInputs=True)

def annual_image(year_date):
  start = ee.Date.fromYMD(year_date, 1, 1)
  end = ee.Date.fromYMD(year_date, 12, 31)
  date_range = ee.DateRange(start, end)
  name = start.format('YYYY_MM').cat('_to_').cat(end.format('YYYY_MM'))
  return modis \
        .filterDate(date_range) \
        .select(band_layer) \
        .reduce(reducer = reducer_list) \
        .multiply(0.0001) \
        .clip(aoi) \
        .set({'name': name})

annual_image = year_list.map(annual_image)

## Check an element of list
year_mosaic  = ee.Image(annual_image.get(2))
label = ee.String(year_mosaic.get('name')).getInfo()
print(label)
print(year_mosaic.getInfo())

## Check a reducer band
band_select = ".*" + img_col_reducer
print(band_select)
print(year_mosaic.select(band_select).getInfo())

In [ ]:
def annual_seasonal_image(year_date, season_start, season_end):
  start = ee.Date.fromYMD(year_date, season_start, 1)
  end = ee.Date.fromYMD(year_date, season_end, 30)
  date_range = ee.DateRange(start, end)
  season_label = "dry" if season_start == 7 else "wet"
  return modis \
        .filterDate(date_range) \
        .select(band_layer) \
        .reduce(reducer = reducer_list) \
        .multiply(0.0001) \
        .clip(aoi) \
        .set({'season': season_label,
              'year': year_date})

def map_seasonal_dry(year):
  return annual_seasonal_image(year, dry_start, dry_end)

annual_dry = year_list.map(map_seasonal_dry)

def map_seasonal_rain(year):
  return annual_seasonal_image(year, rain_start, rain_end)

annual_rain = year_list.map(map_seasonal_rain)

## Export data - create task

`filenamePrefix` should be in format: place_layer_timeperiod

In [ ]:
# Annual images
for i in  range(ee.List.length(annual_image).getInfo()):

  band_select = ".*" + img_col_reducer
  output_img =  ee.Image(annual_image.get(i))
  output_img = output_img.select(band_select)
  output_name = f"{layer_name}_{aoi_name}_{img_col_reducer}_{ee.String(output_img.get('name')).getInfo()}"

  task = ee.batch.Export.image.toDrive(image = output_img,
                                     region = aoi.geometry(),
                                     description = "EXPORT IMAGE TO DRIVE",
                                     folder = "GEE_exports",
                                     fileNamePrefix = output_name,
                                     scale = 250,
                                     maxPixels = 10e12,
                                     crs = "EPSG:4326"
                                     )
  task.start()
  print("STARTED TASK ", i+1)

In [ ]:
# Seasonal images - DRY
for i in range(ee.List.length(annual_dry).getInfo()):
  band_select = ".*" + img_col_reducer
  output_img =  ee.Image(annual_dry.get(i))
  output_img = output_img.select(band_select)
  output_name = f"{layer_name}_{aoi_name}_{img_col_reducer}_{ee.String(output_img.get('year')).getInfo()}_{ee.String(output_img.get('season')).getInfo()}"

  task = ee.batch.Export.image.toDrive(image = output_img,
                                     region = aoi.geometry(),
                                     description = "EXPORT IMAGE TO DRIVE",
                                     folder = "GEE_exports",
                                     fileNamePrefix = output_name,
                                     scale = 250,
                                     maxPixels = 10e12,
                                     crs = "EPSG:4326"
                                     )
  task.start()
  print("STARTED TASK ", "DRY ", i+1)

In [ ]:
# Seasonal images - RAIN
for i in range(ee.List.length(annual_rain).getInfo()):
  band_select = ".*" + img_col_reducer
  output_img =  ee.Image(annual_rain.get(i))
  output_img = output_img.select(band_select)
  output_name = f"{layer_name}_{aoi_name}_{img_col_reducer}_{ee.String(output_img.get('year')).getInfo()}_{ee.String(output_img.get('season')).getInfo()}"

  task = ee.batch.Export.image.toDrive(image = output_img,
                                     region = aoi.geometry(),
                                     description = "EXPORT IMAGE TO DRIVE",
                                     folder = "GEE_exports",
                                     fileNamePrefix = output_name,
                                     scale = 250,
                                     maxPixels = 10e12,
                                     crs = "EPSG:4326"
                                     )
  task.start()
  print("STARTED TASK ", "RAIN ", i+1)

## Check task status

[List](https://developers.google.com/earth-engine/guides/processing_environments#list-of-task-states) of task status messages (state field)


In [ ]:
tasks = ee.batch.Task.list()
for task in tasks[0:ee.List.length(year_list).getInfo()]:
  task_id = task.status()['id']
  task_state = task.status()['state']
  print(task_id, task_state)

## Mount and unmount Google Drive


In [ ]:
# Mount
#from google.colab import drive

#ROOT = '/content/drive'
#drive.mount(ROOT, force_remount=True)


In [ ]:
# Unmount
#drive.flush_and_unmount()